## reference: https://www.tensorflow.org/tutorials/text/text_generation

In [33]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM
import numpy as np
import os
import time

# Load the data

In [34]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# Read the data

In [35]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

# Take a look at the first 250 characters in text
print('\n\n' + text[:250])

Length of text: 1115394 characters


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [36]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

65 unique characters


# Vectorize the text

## Before training, you need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [37]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

## Now you have an integer representation for each character. Notice that you mapped the character as indexes from 0 to len(unique).

In [38]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [39]:
# Show how the first 13 characters from the text are mapped to integers
print('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


# The prediction task

## Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

## Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

# Create training examples and targets

## Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

## For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

## So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

## To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [40]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


## The batch method lets us easily convert these individual characters to sequences of the desired size.

In [41]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


## For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [42]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Print the first example input and target values:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


## Each index of these vectors is processed as a one time step. For the input at time step 0, the model receives the index for "F" and tries to predict the index for "i" as the next character. At the next timestep, it does the same thing but the RNN considers the previous step context in addition to the current input character.

In [43]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


# Create training batches

## You used tf.data to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [44]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

# Build The Model

## Use tf.keras.Sequential to define the model. For this simple example three layers are used to define our model:

    - tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;
    - tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)
    - tf.keras.layers.Dense: The output layer, with vocab_size outputs.

In [45]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [46]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [47]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

# Try the model

## Now run the model to see that it behaves as expected.
## First check the shape of the output:

In [48]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


## In the above example the sequence length of the input is 100 but the model can be run on inputs of any length:

In [49]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


## To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

## Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.

## Try it for the first example in the batch:

In [50]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

## This gives us, at each timestep, a prediction of the next character index:

In [51]:
sampled_indices

array([17,  0, 26, 48, 54, 20, 21, 34, 35,  3, 32,  9, 24, 64, 18,  5, 12,
        1, 39, 33, 41, 17, 44, 21, 51, 36,  9, 56, 50, 61, 52, 62, 26, 62,
       52, 34, 23, 41,  1,  0, 51, 59, 60, 14,  7, 11, 24, 25, 52, 50, 57,
       33,  3, 31, 57,  9, 50, 32, 26, 58,  0, 37, 27, 40, 31,  7, 62, 58,
       10, 59, 54, 37,  1, 31, 31, 27, 13, 15,  4, 55, 51, 26, 32, 56, 59,
        3, 64, 53, 17, 33, 45, 50,  8, 59, 51, 32,  3, 23, 21, 21])

## Decode these to see the text predicted by this untrained model:

In [52]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "hang 'em!\nI would they would forget me, like the virtues\nWhich our divines lose by 'em.\n\nMENENIUS:\nY"

Next Char Predictions: 
 "E\nNjpHIVW$T3LzF'? aUcEfImX3rlwnxNxnVKc \nmuvB-;LMnlsU$Ss3lTNt\nYObS-xt:upY SSOAC&qmNTru$zoEUgl.umT$KII"


# Train the model

## At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

# Attach an optimizer, and a loss function

## The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.
## Because your model returns logits, you need to set the from_logits flag.

In [53]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1738677


## Configure the training procedure using the tf.keras.Model.compile method. Use tf.keras.optimizers.Adam with default arguments and the loss function.

In [54]:
model.compile(optimizer='adam', loss=loss)

# Configure checkpoints

## Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [55]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

# Execute the training

## To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [56]:
EPOCHS = 30
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 9s 54ms/step - loss: 2.6494
Epoch 2/30
172/172 [==============================] - 9s 55ms/step - loss: 1.9504
Epoch 3/30
172/172 [==============================] - 10s 56ms/step - loss: 1.6855
Epoch 4/30
172/172 [==============================] - 10s 56ms/step - loss: 1.5408
Epoch 5/30
172/172 [==============================] - 10s 57ms/step - loss: 1.4533
Epoch 6/30
172/172 [==============================] - 10s 57ms/step - loss: 1.3933
Epoch 7/30
172/172 [==============================] - 10s 58ms/step - loss: 1.3480
Epoch 8/30
172/172 [==============================] - 10s 59ms/step - loss: 1.3092
Epoch 9/30
172/172 [==============================] - 10s 59ms/step - loss: 1.2754
Epoch 10/30
172/172 [==============================] - 10s 59ms/step - loss: 1.2416
Epoch 11/30
172/172 [==============================] - 10s 59ms/step - loss: 1.2094
Epoch 12/30
172/172 [==============================] - 10s 59ms/step - loss: 1.1780
Epo

# Generate text

## Restore the latest checkpoint
## To keep this prediction step simple, use a batch size of 1.

## Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

## To run the model with a different batch_size, you need to rebuild the model and restore the weights from the checkpoint.

In [57]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [58]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [59]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


# The prediction loop

## The following code block generates the text:

    - Begin by choosing a start string, initializing the RNN state and setting the number of characters to generate.

    - Get the prediction distribution of the next character using the start string and the RNN state.

    - Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

    - The RNN state returned by the model is fed back into the model so that it now has more context, instead of only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.

In [60]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [61]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: No, good satisfect, being transported
From these no more must help you to your foundave
More another crown,
Go Henry and his bitter sweet-converner sickn, and a
king her; then But with a little in the duke
Of drams are backs; and I have seen them, next thy hopes, you wondrous less,
That mock'd thee--respected and mis'd, whom I drown'd them not, since they saying
Than brought me in me more abstars hath
wherewithat I cannot, how hear you?
Do you not call thee so?

Second Messenger:
By heaven, for George, for all his fault!
Such as she is a man's pperses, therein last degrees ale.
Then, Lord Of Gloudements, lawful come unto my soul,
And hath been faults,
I have no pity under his doing: this
was but boot.
Here comes the rebellior now?

LUCIO:
Thou liest! what, lad was on me in his life, for many years' kindness,
And craves to beat me crave by the rough on me?
I am too offer'd: crange'd Veronaxamp will I wind
on the sun, do absed thee--
But O, this is the patricionge from the act
Tha